In [ ]:
import subprocess
from dask_cuda import LocalCUDACluster
from dask.distributed import Client, progress
import dask_cudf
import cudf
import dask.dataframe as dd

In [ ]:
import os
os.environ["PATH"] += os.pathsep + '/usr/bin/dot '

In [ ]:
cluster = LocalCUDACluster(dashboard_address=':8902')
client = Client(cluster)
client

In [ ]:
%%time
df = cudf.read_parquet('covid.gzip')
df = df.dropna()
df.loc[df['sex']=='Male']

В dask-cudf применяются ленивые вычисления. То есть мы что-то делаем только тогда, когда это необходимо, например, если вы просите вернуть какой-то ответ.

In [ ]:
%time ddf = dask_cudf.read_parquet('covid.gzip')

In [ ]:
ddf

In [ ]:
gdf = ddf.dropna()

Можно визуализировать граф, котоырй будет исполняться

In [ ]:
gdf.visualize(format='svg')

In [ ]:
male_gdf = gdf.loc[gdf['sex']=='Male']
male_gdf.visualize(format='svg')

А что если я хочу, чтобы уже что-то посчиталось предварительно, пока я буду дальше код писать?

In [ ]:
copy_male_gdf = male_gdf.copy()
copy_male_gdf.visualize(format='svg')

Строчкой ниже мы попросили посчитать все и положить это в память. Действие особенно полезно при использовании распределенных систем,
поскольку результаты будут храниться в распределенной памяти, а не возвращаться в один локальный процесс

In [ ]:
copy_male_gdf = copy_male_gdf.persist()
copy_male_gdf.visualize(format='svg')

Выполнили compute = сложили данные на одну видеокарту и тип данных уже cudf

In [ ]:
%time male_gdf.compute()

In [ ]:
male_gdf = male_gdf.compute()

In [ ]:
male_gdf['age_group'].value_counts() / male_gdf.shape[0]

### В целом все очень похоже на cudf. Но еще пара моментов

In [ ]:
from cudf.datasets import randomdata

df = randomdata(nrows=10, dtypes={'a':float, 'b':float}, seed=12)
df.mean()

Перевод из cudf в dask-cudf

In [ ]:
ddf = dask_cudf.from_cudf(df, npartitions=2)

In [ ]:
type(ddf)

In [ ]:
ddf.mean().compute()

В dask-cudf пока нельзя применять метод applymap к отдельному столбцу напрямую, можно только к каждой партиции через map_partitions.

In [ ]:
def add_ten(num):
    return num + 10

было

In [ ]:
df['a'].applymap(add_ten)

стало

In [ ]:
ddf['a'].map_partitions(add_ten).compute()

Транспонировать партицированные данные пока тоже нельзя, придется сначала выполнить медот compute(), но если в одну видеокарту не влезет, то не выйдет (( А еще если есть переменные типа string, то также не выйдет транспонировать

In [ ]:
ddf.T

In [ ]:
ddf.compute().T

In [ ]:
type(ddf.compute())

# Домашнее задание

Сравните среднюю вероятность смерти мужчин и женщин по группам возростов на основе столбца death_ind. Тоже самое проделайте для вероятности госпитализации, преобразовав переменную hosp_yn, как мы сделали с переменной death_yn. Используйте cudf и сохраните результат на диск.
Как и в ноутбуке с cudf, только теперь с несколькими GPU=)